In [2]:
!pip install langchain_openai

Looking in indexes: https://mirrors.ustc.edu.cn/pypi/web/simple
  Using cached https://mirrors.bfsu.edu.cn/pypi/web/packages/a2/86/05012e5276dde3004cfb0cb383f56983d3adcb5bc8ce13ebf21ce3ea3d26/langchain_openai-0.0.5-py3-none-any.whl (29 kB)
  Using cached https://mirrors.bfsu.edu.cn/pypi/web/packages/37/34/f3c3d6bdc3eebf1b6a7c696dd6f934630af6cf5250cec099edf117cd3b53/openai-1.11.1-py3-none-any.whl (226 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.4 MB/s eta 0:00:00a 0:00:01
  Using cached https://mirrors.bfsu.edu.cn/pypi/web/packages/12/b3/231ffd4ab1fc9d679809f356cebee130ac7daa00d6d6f3206dd4fd137e9e/distro-1.9.0-py3-none-any.whl (20 kB)
  Using cached https://mirrors.bfsu.edu.cn/pypi/web/packages/39/9b/4937d841aee9c2c8102d9a4eeb800c7dad25386caabb4a1bf5010df81a57/httpx-0.26.0-py3-none-any.whl (75 kB)
  Using cached https://mirrors.bfsu.edu.cn/pypi/web/packages/56/ba/78b0a99c4da0ff8b0f59defa2f13ca4668189b134bd9840b6202a93d9a0f/httpcore-1.0.2-py3-none-any.whl (76 kB)

[no

In [3]:
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 文本摘要模版
abstract_template = """
需要实现文本摘要功能，即输入一段文本，输出这段文本的简短摘要。

用户输入文本内容为：
{input}
"""

# 翻译模版
translation_template = """
你的功能是实现翻译功能，即输入一段文本和目标语言，输出这段文本的翻译。

用户输入文本为：
{input}

目标语言：
英文
"""


prompt_infos = [
    {
        "name": "文本摘要",
        "description": "输入一段文本，输出这段文本的摘要。",
        "prompt_template": abstract_template,
    },
    {
        "name": "翻译",
        "description": "输入一段文本和目标语言，输出这段文本的翻译。",
        "prompt_template": translation_template,
    },
]

#创建一个list储存对应的子链名称和描述
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]


#把描述连接成一个str
destinations_str = "\n".join(destinations)

from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)

llm = ChatOpenAI(temperature=0, openai_api_key="sk-****")
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

# 首先，创建一个候选链，包含所有的下游子链
candadite_chains = {}

# 遍历路由目录，生成各子链并放入候选链字典
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    candadite_chains[name] = chain

# 生成默认链
default_chain = ConversationChain(llm=llm, output_key="text")

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=candadite_chains,
    default_chain=default_chain,
    verbose=True,
)



In [4]:
print(chain.run("帮忙翻一下：过中国年，喝王老吉"))


/home/xuwei/jupyterlab/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/home/xuwei/jupyterlab/venv/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(




> Entering new MultiPromptChain chain...
翻译: {'input': '帮忙翻一下：过中国年，喝王老吉'}
> Finished chain.
Translation:
Help translate: During Chinese New Year, drink Wanglaoji.


In [5]:
print(chain.run("""
对以下内容总结：
OLTP 系统通常面向最终用户，这意味着系统可能会收到大量的请求。为了处理负载，应用程序在每个查询中通常只访问少量的记录。应用程序使用某种键来请求记录，存储引擎使用索引来查找所请求的键的数据。硬盘查找时间往往是这里的瓶颈。
数据仓库和类似的分析系统会低调一些，因为它们主要由业务分析人员使用，而不是最终用户。它们的查询量要比 OLTP 系统少得多，但通常每个查询开销高昂，需要在短时间内扫描数百万条记录。硬盘带宽（而不是查找时间）往往是瓶颈，列式存储是针对这种工作负载的日益流行的解决方案。
"""))



> Entering new MultiPromptChain chain...


/home/xuwei/jupyterlab/venv/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


文本摘要: {'input': 'OLTP 系统通常面向最终用户，这意味着系统可能会收到大量的请求。为了处理负载，应用程序在每个查询中通常只访问少量的记录。应用程序使用某种键来请求记录，存储引擎使用索引来查找所请求的键的数据。硬盘查找时间往往是这里的瓶颈。数据仓库和类似的分析系统会低调一些，因为它们主要由业务分析人员使用，而不是最终用户。它们的查询量要比 OLTP 系统少得多，但通常每个查询开销高昂，需要在短时间内扫描数百万条记录。硬盘带宽（而不是查找时间）往往是瓶颈，列式存储是针对这种工作负载的日益流行的解决方案。'}
> Finished chain.
文本摘要：OLTP系统处理大量请求，每次查询访问少量记录，使用索引查找数据。数据仓库查询少但开销大，需要扫描数百万记录，列式存储是解决方案。
